In [ ]:
import warnings
warnings.simplefilter(action = 'ignore')
warnings.filterwarnings(action = 'ignore')
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import logging

# Para evitar excesso de textos quando rodar o treinamento
logging.getLogger("transformers").setLevel(logging.ERROR)
os.environ['TRANSFORMERS_VERBOSITY'] = 'error' 

In [ ]:
# Execute esse bloco apenas se estiver usando o Google Colab para execução --------------------------
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Acesse o Hugging Face e crie uma chave para poder usar alguns modelos, como o LLama e o Gemma -------------------
from huggingface_hub import login
login('**************************************')

In [ ]:
# Carregando o dataset ----------------------------------
pasta = '/content/drive/My Drive/Mestrado/Public_Contracts/'

df = pd.read_parquet(pasta + 'Notas_Fiscais_Itens_2023.parquet')
df.head()

,natureza_operacao,data_emissao,cnpj_cpf_emitente,razao_social_emitente,uf_emitente,mun_emitente,nome_destinatario,descricao_produto,ncm_produto,quantidade,valor_unitario,valor_total,classe,text
0,Outra saida merc./prest.serv. nao especif.,2023-01-01,07432517001847,SIMPRESS COMERCIO LOCACAO E SERVICOS LTDA,SC,ITAJAI,MINISTERIO DA JUSTICA E SEGURANCA PUBLICA,MLTD201LXAZ CARTUCHO DE TONER PRETO 20K PAGINAS,Cartuchos de revelador (toners),1.0,169.58,169.58,0,[CLS] Natureza da operacao: Outra saida merc./...
1,Remessa de bem p/conta contrato de comodato ou...,2023-01-01,07432517001847,SIMPRESS COMERCIO LOCACAO E SERVICOS LTDA,SC,ITAJAI,UNIVERSIDADE FEDERAL RURAL DO RIO DE JANEIRO,HP GABINETE METALICO,Outras partes e acessórios para aparelhos de f...,1.0,560.88,560.88,0,[CLS] Natureza da operacao: Remessa de bem p/c...
2,VENDA DE MARCADORIA ADQUIRIDA DE TERCEIROS,2023-01-01,34849096000189,"SEEK COMERCIO DE LIVROS, JORNAIS E REVISTAS LTDA",RJ,RIO DE JANEIRO,UNIVERSIDADE FEDERAL DO DELTA DO PARNAIBA UFDPAR,METODOS QUANTITATIVOS APLICADOS A CONTABILIDADE,"Outros livros, brochuras e impressos semelhantes",12.0,82.60,991.20,0,[CLS] Natureza da operacao: VENDA DE MARCADORI...
3,VENDA DE MARCADORIA ADQUIRIDA DE TERCEIROS,2023-01-01,34849096000189,"SEEK COMERCIO DE LIVROS, JORNAIS E REVISTAS LTDA",RJ,RIO DE JANEIRO,UNIVERSIDADE FEDERAL DO DELTA DO PARNAIBA UFDPAR,GESTAO ESTRATEGICA DE ARMAZENAMENTO,"Outros livros, brochuras e impressos semelhantes",12.0,78.40,940.80,0,[CLS] Natureza da operacao: VENDA DE MARCADORI...
4,VENDA DE MARCADORIA ADQUIRIDA DE TERCEIROS,2023-01-01,34849096000189,"SEEK COMERCIO DE LIVROS, JORNAIS E REVISTAS LTDA",RJ,RIO DE JANEIRO,UNIVERSIDADE FEDERAL DO DELTA DO PARNAIBA UFDPAR,RELACOES INTERNACIONAIS DA ASIA E DA AFRICA,"Outros livros, brochuras e impressos semelhantes",12.0,88.20,1058.40,0,[CLS] Natureza da operacao: VENDA DE MARCADORI...


In [ ]:
# Removendo linhas com NA's ---------------------------------
df = df.dropna().reset_index(drop = True)
df = df[['text', 'classe']].rename(columns = {'classe':'label'})
len(df)

5980558

In [ ]:
print('Dados de treino:\n',df.groupby('label', as_index = False).size())

Dados de treino:
    label     size
0      0  5731120
1      1   249438


In [ ]:
# Criando dataset balanceado com N quantidades de cada classe selecionadas de forma random ----------------------
samples_per_class_big = 10025

# Verificar se há exemplos suficientes em cada classe
count_label_0 = df[df['label'] == 0].shape[0]
count_label_1 = df[df['label'] == 1].shape[0]

if count_label_0 >= samples_per_class_big and count_label_1 >= samples_per_class_big:

    df_label_0 = df[df['label'] == 0].sample(n=samples_per_class_big, random_state=42)

    df_label_1 = df[df['label'] == 1].sample(n=samples_per_class_big, random_state=42)

    df_amostra = pd.concat([df_label_0, df_label_1]).reset_index(drop=True)
else:
    
    print("Não há exemplos suficientes em uma das classes para realizar a amostragem desejada.")

In [ ]:
print('Dados de treino:\n',df_amostra.groupby('label', as_index = False).size())

Dados de treino:
    label   size
0      0  10025
1      1  10025


In [ ]:
# Criando dataset balanceado com N exemplos few-shot de cada classe selecionadas de forma random ----------------------
samples_per_class = 25 
df_0 = df_amostra[df_amostra['label'] == 0].sample(n=samples_per_class, random_state=42)
df_1 = df_amostra[df_amostra['label'] == 1].sample(n=samples_per_class, random_state=42)
df_few = pd.concat([df_0, df_1]).sample(frac=1, random_state=42)


In [ ]:
# Remove do df original as linhas selecionadas para few-shot
ids_few = df_few.index
df_test = df_amostra.drop(ids_few)

print("Few-shot (prompt) size:", len(df_few))
print("Test set size:", len(df_test))


Few-shot (prompt) size: 50
Test set size: 20000


In [ ]:
# Gerar um prompt com os exemplos de few-shot ---------------------------
def build_prompt(few_examples, new_text):

    prompt = "Você é um modelo de IA que classifica o texto em 0 ou 1.\n"
    
    prompt += "Abaixo estão alguns exemplos:\n\n"

    for i, row in few_examples.iterrows():

        prompt += f"Exemplo:\nTexto: \"{row['text']}\"\nClassificação: {row['label']}\n\n"

    prompt += "Agora, classifique o texto abaixo:\n"

    prompt += f"Texto: \"{new_text}\"\n\n"

    prompt += "Responda somente com 0 ou 1.\nClassificação:"

    return prompt

# Faz a classificação dos textos ---------------------------------------------
def classify_text_llm(model, tokenizer, few_examples, text,
                      max_new_tokens=10, temperature=0.0):

    prompt = build_prompt(few_examples, text)

    inputs = tokenizer.encode(prompt, return_tensors="pt").to(model.device)

    tokenizer.pad_token_id = tokenizer.eos_token_id

    with torch.no_grad():

        outputs = model.generate(
            inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=False
        )

    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Tentar extrair a resposta final (0 ou 1) do texto gerado
    answer_part = generated.split("Classificação:")[-1].strip()

    # Pegar somente o primeiro caractere que seja '0' ou '1'
    if len(answer_part) > 0:

        # Filtrar apenas o primeiro caractere que seja '0' ou '1'
        if answer_part[0] == '0':

            return 0
        
        elif answer_part[0] == '1':
            
            return 1
        
        # Se não achou nada, retorna None
        else:

            return None  

In [ ]:
warnings.filterwarnings(
    "ignore",
    message="The attention mask and the pad token id were not set."
)

In [ ]:
#model_id = "mistralai/Mistral-7B-v0.1"
#model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
#model_id = "deepseek-ai/DeepSeek-R1"
#model_id = "google/gemma-3-12b-pt"
#model_id = "meta-llama/Llama-3.2-3B"
model_id = "meta-llama/Llama-3.1-8B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto", 
    torch_dtype=torch.float16
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
model.generation_config.pad_token_id = tokenizer.pad_token_id
tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
y_true = []
y_pred = []

for i, row in tqdm(df_test.iterrows()):
    
    label_verdadeiro = row['label']
    texto = row['text']

    # Classificar
    label_predito = classify_text_llm(
        model=model,
        tokenizer=tokenizer,
        few_examples=df_few,
        text=texto
    )

    y_true.append(label_verdadeiro)
    y_pred.append(label_predito)

y_true = np.array(y_true)
y_pred = np.array(y_pred)


20000it [5:23:08,  1.03it/s]


In [ ]:
# Criar a matriz de confusão -----------------------------------
cm = confusion_matrix(y_true, y_pred)
tn, fp, fn, tp = cm.ravel()

print("Matriz de Confusão (tn, fp, fn, tp):")
print(cm)

# [[TN, FP],
# [FN, TP]]

Matriz de Confusão (tn, fp, fn, tp):
[[8958 1017]
 [7799 2226]]


In [ ]:
# Criar a matriz de custo -----------------------------------
cost_FP = 1
cost_FN = 5
cost_TP = 0
cost_TN = 0

total_cost = (fp * cost_FP) + (fn * cost_FN) + (tp * cost_TP) + (tn * cost_TN)
print(f"Custo Total: {total_cost}")

Custo Total: 40012


In [ ]:
# Criar a matriz de custo -----------------------------------
cost_FP = 5
cost_FN = 1
cost_TP = 0
cost_TN = 0

total_cost = (fp * cost_FP) + (fn * cost_FN) + (tp * cost_TP) + (tn * cost_TN)
print(f"Custo Total: {total_cost}")

Custo Total: 12884


In [ ]:
# Avaliar o modelo ------------------------------------------
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

print(f"Acurácia:  {accuracy:.4f}")
print(f"Precisão:  {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")


Acurácia:  0.5592
Precisão:  0.6864
Recall:    0.2220
F1 Score:  0.3355
